In [ ]:
import pandas as pd
import numpy as np
from glob import glob
from tqdm.notebook import tqdm, trange
import os
import datetime

In [ ]:
W, H = 1920, 1080

In [ ]:
files = glob('../runs/detect/predict/labels/*') + glob('../runs/detect/predict[234]/labels/*')

In [ ]:
len(files)

In [ ]:
set([x.split('/')[3] for x in files])

In [ ]:
# files = [x for x in files if 'predict5' not in x and 'predict6' not in x]

In [ ]:
len(files)

In [ ]:
files[:5]

In [ ]:
# filename, camera_id, datetime, frame_id, label_class, x,y,x,y, confidance   #;;;;
coco_cols = ['label_class', 'x', 'y', 'w', 'h', 'confidence']
res = []
for x in tqdm(files):
    bname = os.path.basename(x).replace('.txt', '')
    sname = bname.split('_')

    video_file = '_'.join(sname[:-1]) + '.mkv'
    frame = int(sname[-1])
    camera = sname[3]
    dt = sname[6]
    dt_parsed = datetime.datetime.strptime(dt, "%Y%m%dT%H%M%S")
    
    with open(x, 'r') as file:
        lines = [x.rstrip() for x in file.readlines()]
        for l in lines:
            l_split = l.split(' ')
            curr_data = {v: float(l_split[k]) for k, v in enumerate(coco_cols)}
            curr_data['filename'] = video_file
            curr_data['camera_id'] = camera
            curr_data['datetime'] = dt_parsed
            curr_data['frame_id'] = frame
            res.append(curr_data)

df_res = pd.DataFrame.from_records(res).loc[:, ['filename', 'camera_id', 'datetime', 'frame_id', 'label_class', 'x', 'y', 'w', 'h', 'confidence']]
df_res['label_class'] = df_res['label_class'].astype(int)
df_res

In [ ]:
df_res['frame_id'] = df_res['frame_id'] - 1

In [ ]:
df_res['x'] = (df_res['x'] * W).astype(int)
df_res['y'] = (df_res['y'] * H).astype(int)
df_res['w'] = (df_res['w'] * W).astype(int)
df_res['h'] = (df_res['h'] * H).astype(int)

In [ ]:
df_res.head()

In [ ]:
df_res.shape

In [ ]:
df_res.to_csv('../annotations/yolov8x_full_data_train_video_labels.csv', sep=';')